## Best Practices of Class Design

How do you design classes for inheritance? Does Python have private attributes? Is it possible to control attribute access? You'll find answers to these questions (and more) as you learn class design best practices.

### Square and rectangle
The classic example of a problem that violates the Liskov Substitution Principle is the Circle-Ellipse problem, sometimes called the Square-Rectangle problem.

By all means, it seems like you should be able to define a class Rectangle, with attributes h and w (for height and width), and then define a class Square that inherits from the Rectangle. After all, a square "is-a" rectangle!

Unfortunately, this intuition doesn't apply to object-oriented design.

In [ ]:
# Define a Rectangle class
class Rectangle:
    def __init__(self, h, w):
        self.h = h
        self.w = w

# Define a Square class
class Square(Rectangle):
    def __init__(self, w):
        self.h = w
        self.w = w

In [ ]:
# A Square inherited from a Rectangle will always have both the h and w attributes, 
# but we can't allow them to change independently of each other.

class Rectangle:
    def __init__(self, w,h):
      self.w, self.h = w,h
      
# Define set_h to set h       
    def set_h(self, h):
      self.h = h

# Define set_w to set w
    def set_w(self, w):
      self.w = w   
      
class Square(Rectangle):
    def __init__(self, w):
      self.w, self.h = w, w 
      
# Define set_h to set w and h 
    def set_h(self, h):
      self.h = h
      self.w = h
      
# Define set_w to set w and h 
    def set_w(self, w):
      self.w = w   
      self.h = w  
      

### Using internal attibutes
In this exercise, you'll return to the BetterDate class of Chapter 2. Your date class is better because it will use the sensible convention of having exactly 30 days in each month.

You decide to add a method that checks the validity of the date, but you don't want to make it a part of BetterDate public interface.

The class BetterDate is available in the script pane.

In [1]:
# MODIFY to add class attributes for max number of days and months
class BetterDate:
    _MAX_DAYS = 30
    __MAX_MONTH = 12
    def __init__(self, year, month, day):
      self.year, self.month, self.day = year, month, day
      
    @classmethod
    def from_str(cls, datestr):
        year, month, day = map(int, datestr.split("-"))
        return cls(year, month, day)
    
    # Add _is_valid() checking day and month values
    def _is_valid(self):
        if (self.day <= BetterDate._MAX_DAYS) & (self.month <= BetterDate.__MAX_MONTH):
            print(True)
        else: 
            print(False)
         
bd1 = BetterDate(2020, 4, 30)
print(bd1._is_valid())

bd2 = BetterDate(2020, 6, 45)
print(bd2._is_valid())

True
None
False
None


### Create and set properties
There are two parts to defining a property:

first, define an "internal" attribute that will contain the data;
then, define a @property-decorated method whose name is the property name, and that returns the internal attribute storing the data.

If you'd also like to define a custom setter method, there's an additional step:

define another method whose name is exactly the property name (again), and decorate it with @prop_name.setter where prop_name is the name of the property. The method should take two arguments -- self (as always), and the value that's being assigned to the property.

In this exercise, you'll create a balance property for a Customer class - a better, more controlled version of the balance attribute that you worked with before.

In [3]:
# Create a Customer class
class Customer:
    def __init__(self, name, new_bal):
        self.name = name
        if new_bal < 0:
            raise ValueError
        else:
            self._balance = new_bal

In [4]:
# Add a method balance() with a @property decorator that returns the _balance attribute.

class Customer:
    def __init__(self, name, new_bal):
        self.name = name
        if new_bal < 0:
           raise ValueError("Invalid balance!")
        self._balance = new_bal  
    
    # Add a decorated balance() method returning _balance
    @property
    def balance(self):
        return self._balance

In [5]:
# Define another balance() method to serve as a setter, with the appropriate decorator and an additional parameter

class Customer:
    def __init__(self, name, new_bal):
        self.name = name
        if new_bal < 0:
           raise ValueError("Invalid balance!")
        self._balance = new_bal  

    # Add a decorated balance() method returning _balance        
    @property
    def balance(self):
        return self._balance
     
    # Add a setter balance() method
    @balance.setter
    def balance(self, balance):
        # Validate the parameter value
        if balance < 0:
           raise ValueError("Invalid balance!")
        self._balance = balance
        
        # Print "Setter method is called"
        print("Setter method is called")
        

In [6]:
# Create a Customer named Belinda Lutz with the balance of 2000 and save it as cust.
 
cust = Customer('Belinda Lutz', 2000)

# Assign 3000 to the balance property
cust.balance = 3000

# Print the balance property
print(cust.balance)

Setter method is called
3000


### Read-only properties
The LoggedDF class from Chapter 2 was an extension of the pandas DataFrame class that had an additional created_at attribute that stored the timestamp when the DataFrame was created, so that the user could see how out-of-date the data is.

But that class wasn't very useful: we could just assign any value to created_at after the DataFrame was created, thus defeating the whole point of the attribute! Now, using properties, we can make the attribute read-only.

In [7]:
import pandas as pd
from datetime import datetime

# MODIFY the class to turn created_at into a read-only property
class LoggedDF(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        pd.DataFrame.__init__(self, *args, **kwargs)
        self.created_at = datetime.today()

    def to_csv(self, *args, **kwargs):
        temp = self.copy()
        temp["created_at"] = self.created_at
        pd.DataFrame.to_csv(temp, *args, **kwargs)   

ldf = LoggedDF({"col1": [1,2], "col2":[3,4]}) 

# Put into try-except block to catch AtributeError and print a message
ldf.created_at = '2035-07-13'